In [241]:
import os
import shutil 
import csv
import numpy as np
#import numpy as np
#import pandas as pd
#from sklearn.preprocessing import StandardScaler
from utils import *

In [242]:
band_name = 'v8'  
window_size = 6 

In [265]:
path = 'sources/Ini_sources/'
#print(os.getcwd())
data_1 = make_sequences_y1(band_name, path, band_name)
print(len(data_1))

10


In [266]:
''' Writing out initial sequences
file_ini = open('sources/Ini_sources/sequences_'+band_name, 'w')
for line in data_1:
    file_ini.write(str(line) +'\n')
    
file_ini.close()
'''

" Writing out initial sequences\nfile_ini = open('sources/Ini_sources/sequences_'+band_name, 'w')\nfor line in data_1:\n    file_ini.write(str(line) +'\n')\n    \nfile_ini.close()\n"

In [267]:
from decimal import Decimal, ROUND_HALF_UP, getcontext
getcontext().rounding = ROUND_HALF_UP
l = len(data_1)
h = int(round(Decimal(l)/2,0))
i = int(round(Decimal(h)/2,0))
indexes = [i, l-(h-i)]
print(indexes)

[3, 8]


In [268]:
train = data_1[:indexes[0]] + data_1[indexes[1]:]
test  = data_1[indexes[0]:indexes[1]]
print(len(train))

5


In [71]:
data_test_1 = get_x(test, window_size=window_size)

file_csv = open('sources/test_y1_'+band_name+'.csv', 'w', newline='')
filewriter = csv.writer(file_csv, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
filewriter.writerow([1,2,3,4,5,6,'Y'])
for i in range(data_test_1.shape[0]):
    filewriter.writerow(data_test_1[i,:])
file_csv.close()

# Creating additional samples for train set

In [269]:
data = get_x(train, window_size=window_size).tolist()
print(len(data))

61


# Negative sequence

In [270]:
data_add = []
for el in data:
    seq = []
    for cell in el[:-1]:
        seq.append(-cell)
    seq.append(1.0)
    data_add.append(seq)

for el in data_add:
    data.append(el)
print(len(data))

122


In [271]:
data_add = []
for el in data:
    el = el[:-1]
    el.reverse()
    el.append(1.)
    data_add.append(el)

for el in data_add:
    data.append(el)
print(len(data))

244


In [278]:
def create_deltas(line, y_label=False):
    if y_label == True:
        line = line[:-1]
    deltas = [0]
    for i in range(len(line)-1):
        delta = line[i] - line[i+1]
        deltas.append(delta)
    return deltas

def create_shift_in_deltas(data, shift):
    data_add = []

    for line in data:
        deltas = create_deltas(line, y_label=True)
        new = [line[0]]
        ini = line[0]
        for i in range(1, len(line[:-1])):
            ini = ini - deltas[i] - shift
            new.append(ini)
        new.append(1.)
        data_add.append(new)
    
    return data_add

In [276]:
shifts = [i for i in np.arange(-0.015, 0.015, 0.0005) if abs(i) > 0.00001]
data_add_res = []
for shift in shifts:
    data_add = create_shift_in_deltas(data, shift)
    data_add_res += data_add
print(len(data_add_res))

#print(data_add_res[2715:2720])

886416
[[3.4878740000000334, 3.4870459999999865, 3.484436000000005, 3.4807840000000394, 3.476576999999997, 3.472749999999905, 1.0], [3.4620459999999866, 3.459436000000005, 3.4557840000000395, 3.4515769999999972, 3.447749999999905, 3.4449750000001127, 1.0], [3.434436000000005, 3.4307840000000396, 3.4265769999999973, 3.422749999999905, 3.419975000000113, 3.418734999999969, 1.0], [3.4057840000000397, 3.4015769999999974, 3.3977499999999052, 3.394975000000113, 3.3937349999999693, 3.392734999999902, 1.0], [3.3765769999999975, 3.3727499999999053, 3.369975000000113, 3.3687349999999694, 3.367734999999902, 3.369011, 1.0]]


In [279]:
data_add = []
for line in data:
    deltas = create_deltas(line, y_label=True)
    #print(line)
    #print('d ', deltas)
    if all(delta <= 0.1 for delta in deltas) is True:
        cell_max = max(map(abs,deltas))
        cell_min = min(map(abs,deltas[1:]))
        while (cell_max < 0.05):
            deltas = [d*10 for d in deltas]
            print('d ', deltas)
            cell_max *= 10
            new = []
            ini = line[0]
            for i in range(len(deltas)):
                ini = ini - deltas[i]
                new.append(ini)
            new.append(1.)
            #print('n ',new)
            data_add.append(new)
            
            if cell_min >= 0.002:
                deltas2 = [d/2 for d in deltas]
                new = []
                #print('1', deltas)
                #print('2',deltas2)
                ini = line[0]
                for i in range(len(deltas2)):
                    ini = ini - deltas2[i]
                    new.append(ini)
                new.append(1.)
                #print('n ',new)
                data_add.append(new)
            
print(len(data_add))

d  [0, 0.021360000000640866, 0.0412400000004709, 0.06022999999913736, 0.06865999999945416, 0.078400000001011]
d  [0, 0.0412400000004709, 0.06022999999913736, 0.06865999999945416, 0.078400000001011, 0.06925999999907617]
d  [0, 0.06022999999913736, 0.06865999999945416, 0.078400000001011, 0.06925999999907617, 0.06099000000062915]
d  [0, 0.06865999999945416, 0.078400000001011, 0.06925999999907617, 0.06099000000062915, 0.06429000000025553]
d  [0, 0.078400000001011, 0.06925999999907617, 0.06099000000062915, 0.06429000000025553, 0.04167000000052212]
d  [0, 0.06925999999907617, 0.06099000000062915, 0.06429000000025553, 0.04167000000052212, 0.029359999998632702]
d  [0, 0.06099000000062915, 0.06429000000025553, 0.04167000000052212, 0.029359999998632702, 0.0179800000000796]
d  [0, 0.06429000000025553, 0.04167000000052212, 0.029359999998632702, 0.0179800000000796, 0.006090000000540385]
d  [0, 0.04167000000052212, 0.029359999998632702, 0.0179800000000796, 0.006090000000540385, -0.010760000000118453

d  [0, -0.15298000000008027, -0.16435999999863338, -0.1766700000005228, -0.1992900000002562, -0.19599000000062983]
d  [0, -0.14109000000054106, -0.15298000000008027, -0.16435999999863338, -0.1766700000005228, -0.1992900000002562]
d  [0, -0.12423999999988222, -0.14109000000054106, -0.15298000000008027, -0.16435999999863338, -0.1766700000005228]
d  [0, -0.11688999999911243, -0.12423999999988222, -0.14109000000054106, -0.15298000000008027, -0.16435999999863338]
d  [0, -0.096980000000495, -0.11688999999911243, -0.12423999999988222, -0.14109000000054106, -0.15298000000008027]
d  [0, -0.09365999999978225, -0.096980000000495, -0.11688999999911243, -0.12423999999988222, -0.14109000000054106]
d  [0, -0.08238000000097667, -0.09365999999978225, -0.096980000000495, -0.11688999999911243, -0.12423999999988222]
d  [0, -0.15451000000076487, -0.15387999999894486, -0.15522000000010916, -0.15158000000020433, -0.15010999999936825]
d  [0, -0.15263999999980182, -0.15451000000076487, -0.15387999999894486, -0

d  [0, -0.06875999999953031, -0.049770000000863845, -0.04134000000054705, -0.031599999998990214, -0.040740000000925036]
d  [0, -0.049770000000863845, -0.04134000000054705, -0.031599999998990214, -0.040740000000925036, -0.049009999999372056]
d  [0, -0.49770000000863845, -0.4134000000054705, -0.31599999998990214, -0.40740000000925036, -0.49009999999372056]
d  [0, -0.04134000000054705, -0.031599999998990214, -0.040740000000925036, -0.049009999999372056, -0.045709999999745676]
d  [0, -0.4134000000054705, -0.31599999998990214, -0.40740000000925036, -0.49009999999372056, -0.45709999999745676]
d  [0, -0.031599999998990214, -0.040740000000925036, -0.049009999999372056, -0.045709999999745676, -0.06832999999947909]
d  [0, -0.040740000000925036, -0.049009999999372056, -0.045709999999745676, -0.06832999999947909, -0.0806400000013685]
d  [0, -0.049009999999372056, -0.045709999999745676, -0.06832999999947909, -0.0806400000013685, -0.09201999999992161]
d  [0, -0.045709999999745676, -0.068329999999479

d  [0, -0.3782700000009198, -0.3677499999979217, -0.3524000000014338, -0.350000000000672, -0.3272399999990183]
d  [0, -0.3677499999979217, -0.3524000000014338, -0.350000000000672, -0.3272399999990183, -0.3140600000004623]
d  [0, -0.3524000000014338, -0.350000000000672, -0.3272399999990183, -0.3140600000004623, -0.2994099999976596]
d  [0, -0.350000000000672, -0.3272399999990183, -0.3140600000004623, -0.2994099999976596, -0.3118000000016208]
d  [0, -0.3272399999990183, -0.3140600000004623, -0.2994099999976596, -0.3118000000016208, -0.25393999999991035]
d  [0, -0.3140600000004623, -0.2994099999976596, -0.3118000000016208, -0.25393999999991035, -0.25182000000071536]
d  [0, -0.2994099999976596, -0.3118000000016208, -0.25393999999991035, -0.25182000000071536, -0.2773300000011103]
d  [0, -0.3118000000016208, -0.25393999999991035, -0.25182000000071536, -0.2773300000011103, -0.25403999999738414]
d  [0, -0.2501399999992726, -0.2673100000002604, -0.2815400000007129, -0.3259399999988899, -0.356670

d  [0, 0.1468000000016234, 0.08893999999991298, 0.08682000000071799, 0.11233000000111293, 0.08903999999738677]
d  [0, 0.08513999999927524, 0.10231000000026302, 0.11654000000071552, 0.16093999999889252, 0.19167000000088752]
d  [0, 0.10231000000026302, 0.11654000000071552, 0.16093999999889252, 0.19167000000088752, 0.21844999999921288]
d  [0, 0.11654000000071552, 0.16093999999889252, 0.19167000000088752, 0.21844999999921288, 0.26090000000003943]
d  [0, 0.16093999999889252, 0.19167000000088752, 0.21844999999921288, 0.26090000000003943, 0.30680000000011365]
d  [0, 0.19167000000088752, 0.21844999999921288, 0.26090000000003943, 0.30680000000011365, 0.30932000000057247]
d  [0, 0.21844999999921288, 0.26090000000003943, 0.30680000000011365, 0.30932000000057247, 0.3881299999982746]
d  [0, 0.26090000000003943, 0.30680000000011365, 0.30932000000057247, 0.3881299999982746, 0.4286300000006138]
d  [0, 0.30680000000011365, 0.30932000000057247, 0.3881299999982746, 0.4286300000006138, 0.4751100000014974]

...

d  [0, -0.02840000000101206, -0.018659999999455223, -0.010229999999138428, 0.008759999999528034, 0.028639999999358068]
d  [0, -0.2840000000101206, -0.18659999999455223, -0.10229999999138428, 0.08759999999528034, 0.2863999999935807]
d  [0, -0.019259999999077237, -0.02840000000101206, -0.018659999999455223, -0.010229999999138428, 0.008759999999528034]
d  [0, -0.19259999999077237, -0.2840000000101206, -0.18659999999455223, -0.10229999999138428, 0.08759999999528034]
d  [0, -0.010990000000630218, -0.019259999999077237, -0.02840000000101206, -0.018659999999455223, -0.010229999999138428]
d  [0, -0.10990000000630218, -0.19259999999077237, -0.2840000000101206, -0.18659999999455223, -0.10229999999138428]
d  [0, -0.014290000000256597, -0.010990000000630218, -0.019259999999077237, -0.02840000000101206, -0.018659999999455223]
d  [0, -0.14290000000256597, -0.10990000000630218, -0.19259999999077237, -0.2840000000101206, -0.18659999999455223]
d  [0, 0.008329999999476811, -0.014290000000256597, -0.0109

d  [0, 0.291800000001623, 0.2794099999976618, 0.2940600000004645, 0.3072399999990205, 0.3300000000006742]
d  [0, 0.23393999999991255, 0.291800000001623, 0.2794099999976618, 0.2940600000004645, 0.3072399999990205]
d  [0, 0.23182000000071756, 0.23393999999991255, 0.291800000001623, 0.2794099999976618, 0.2940600000004645]
d  [0, 0.2573300000011125, 0.23182000000071756, 0.23393999999991255, 0.291800000001623, 0.2794099999976618]
d  [0, 0.23403999999738634, 0.2573300000011125, 0.23182000000071756, 0.23393999999991255, 0.291800000001623]
d  [0, 0.3366700000008871, 0.3059399999988921, 0.2615400000007151, 0.2473100000002626, 0.2301399999992748]
d  [0, 0.36344999999921246, 0.3366700000008871, 0.3059399999988921, 0.2615400000007151, 0.2473100000002626]
d  [0, 0.405900000000039, 0.36344999999921246, 0.3366700000008871, 0.3059399999988921, 0.2615400000007151]
d  [0, 0.4518000000001132, 0.405900000000039, 0.36344999999921246, 0.3366700000008871, 0.3059399999988921]
d  [0, 0.45432000000057204, 0.451

d  [0, -0.06903999999738453, -0.09233000000111069, -0.06682000000071575, -0.06893999999991074, -0.12680000000162117]
d  [0, -0.17167000000088528, -0.1409399999988903, -0.09654000000071328, -0.08231000000026079, -0.065139999999273]
d  [0, -0.19844999999921065, -0.17167000000088528, -0.1409399999988903, -0.09654000000071328, -0.08231000000026079]
d  [0, -0.2409000000000372, -0.19844999999921065, -0.17167000000088528, -0.1409399999988903, -0.09654000000071328]
d  [0, -0.2868000000001114, -0.2409000000000372, -0.19844999999921065, -0.17167000000088528, -0.1409399999988903]
d  [0, -0.28932000000057023, -0.2868000000001114, -0.2409000000000372, -0.19844999999921065, -0.17167000000088528]
d  [0, -0.36812999999827234, -0.28932000000057023, -0.2868000000001114, -0.2409000000000372, -0.19844999999921065]
d  [0, -0.40863000000061156, -0.36812999999827234, -0.28932000000057023, -0.2868000000001114, -0.2409000000000372]
d  [0, -0.45511000000149515, -0.40863000000061156, -0.36812999999827234, -0.289

d  [0, 0.15466999999886877, 0.12701000000049145, 0.11891999999931624, 0.11924000000078205, 0.10903999999924974]
d  [0, 0.1287400000006711, 0.15466999999886877, 0.12701000000049145, 0.11891999999931624, 0.11924000000078205]
d  [0, 0.14651000000014403, 0.1287400000006711, 0.15466999999886877, 0.12701000000049145, 0.11891999999931624]
d  [0, 0.16674000000022726, 0.14651000000014403, 0.1287400000006711, 0.15466999999886877, 0.12701000000049145]
d  [0, 0.17724999999984004, 0.16674000000022726, 0.14651000000014403, 0.1287400000006711, 0.15466999999886877]
d  [0, -0.0910800000002654, -0.059510000000391194, -0.02895999999907861, 0.016849999998949095, 0.040200000000250036]
d  [0, -0.11660999999947297, -0.0910800000002654, -0.059510000000391194, -0.02895999999907861, 0.016849999998949095]
d  [0, -0.13828000000046803, -0.11660999999947297, -0.0910800000002654, -0.059510000000391194, -0.02895999999907861]
d  [0, -0.15609999999981472, -0.13828000000046803, -0.11660999999947297, -0.0910800000002654,

d  [0, 0.17966999999886824, 0.15201000000049092, 0.1439199999993157, 0.14424000000078152, 0.1340399999992492]
d  [0, 0.15374000000067056, 0.17966999999886824, 0.15201000000049092, 0.1439199999993157, 0.14424000000078152]
d  [0, 0.1715100000001435, 0.15374000000067056, 0.17966999999886824, 0.15201000000049092, 0.1439199999993157]
d  [0, 0.19174000000022673, 0.1715100000001435, 0.15374000000067056, 0.17966999999886824, 0.15201000000049092]
d  [0, 0.2022499999998395, 0.19174000000022673, 0.1715100000001435, 0.15374000000067056, 0.17966999999886824]
d  [0, -0.06608000000026593, -0.03451000000039173, -0.003959999999079145, 0.04184999999894856, 0.0652000000002495]
d  [0, -0.0916099999994735, -0.06608000000026593, -0.03451000000039173, -0.003959999999079145, 0.04184999999894856]
d  [0, -0.11328000000046856, -0.0916099999994735, -0.06608000000026593, -0.03451000000039173, -0.003959999999079145]
d  [0, -0.13109999999981525, -0.11328000000046856, -0.0916099999994735, -0.06608000000026593, -0.034

In [275]:
for el in data_add_res:
    data.append(el)

for el in data_add:
    data.append(el)
print(len(data))   

15024


In [25]:
'''
data_add = []
for el in data:
    multiplicate = [2,2.5,3,0.5,0.2,0.1]
    for value in multiplicate:
        line_add = el.copy()
    
        for i in range(len(line_add)-1):
            line_add[i] = round(line_add[i] * value, 3)
        data_add.append(line_add)
        
    increase = [0.5,1,1.5,2,2.5,3,-0.5,-1,-1.5,-2,-2.5,-3]
    for value in increase:
        line_add = el.copy()
    
        for i in range(len(line_add)-1):
            line_add[i] = round(line_add[i] + value, 3)
        data_add.append(line_add)
'''

'\ndata_add = []\nfor el in data:\n    multiplicate = [2,2.5,3,0.5,0.2,0.1]\n    for value in multiplicate:\n        line_add = el.copy()\n    \n        for i in range(len(line_add)-1):\n            line_add[i] = round(line_add[i] * value, 3)\n        data_add.append(line_add)\n        \n    increase = [0.5,1,1.5,2,2.5,3,-0.5,-1,-1.5,-2,-2.5,-3]\n    for value in increase:\n        line_add = el.copy()\n    \n        for i in range(len(line_add)-1):\n            line_add[i] = round(line_add[i] + value, 3)\n        data_add.append(line_add)\n'

In [ ]:
'''
for el in data_add:
    data.append(el)
print(len(data))
'''

# y=0

In [ ]:
def change_one_value_in_sequence(line_ini, delta=0):
    line_add = line_ini.copy()
    line_add[i] = round(line_ini[i-1] + delta, 3)
    line_add[-1] = 0.
    return line_add
    

In [ ]:
data_add = []
for i in range(1, 6):
    for line in data:
        line_add = change_one_value_in_sequence(line, delta=0.002)
        data_add.append(line_add)
        
        line_add = change_one_value_in_sequence(line, delta=-0.002)
        data_add.append(line_add)
        
for line in data_add:
        data.append(line)
        
print(len(data))


# Write out

In [277]:
data_train = np.array(data)

file_csv = open('sources/train_y1_'+band_name+'.csv', 'w', newline='')
filewriter = csv.writer(file_csv, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
filewriter.writerow([1,2,3,4,5,6,'Y'])
for i in range(data_train.shape[0]):
    filewriter.writerow(data_train[i,:])
    '''negative = list(map(lambda x: (-1)*x, data_train_1[i,:-1]))
    negative = np.flip(negative)
    add = data_train_1[i,:].copy()
    add[:-1] = negative
    filewriter.writerow(add)'''
'''for i in range(data_train_1.shape[0]):
    rand = data_train_1[i,:-1].copy()
    np.random.shuffle(rand)
    add = data_train_1[0,:].copy()
    add[:-1] = rand
    add[-1] = 0
    filewriter.writerow(add)
'''
file_csv.close()